# Inference + Intervals


## Notebook Guide

- **Purpose:** Run 24h forecasts with prediction intervals from trained models.
- **Inputs:** features.parquet + artifacts/models/*
- **Outputs:** forecast tables + plots.
- **Run:** Execute cells top‑to‑bottom. If a file is missing, run the earlier pipeline notebook first.


Load model bundles and generate 24h forecasts with intervals.

In [ ]:
import sys, subprocess
from pathlib import Path

print("Python:", sys.executable)
repo_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
subprocess.run(["pip", "install", "-e", str(repo_root)], check=True)


In [ ]:
import pandas as pd
from pathlib import Path
from gridpulse.forecasting.predict import load_model_bundle, predict_next_24h

features_path = Path(repo_root) / "data" / "processed" / "features.parquet"
model_path = Path(repo_root) / "artifacts" / "models" / "lstm_load_mw.pt"

print("Features:", features_path)
print("Model:", model_path)

features = pd.read_parquet(features_path)
bundle = load_model_bundle(model_path)
forecast = predict_next_24h(features, bundle)
forecast


In [ ]:
import pandas as pd
import plotly.express as px

rows = pd.DataFrame({
    "timestamp": forecast["timestamp"],
    "p50": forecast["forecast"],
})
for q, vals in forecast.get("quantiles", {}).items():
    rows[f"q{q}"] = vals

rows["timestamp"] = pd.to_datetime(rows["timestamp"], utc=True)
fig = px.line(rows, x="timestamp", y=[c for c in rows.columns if c != "timestamp"])
fig



In [ ]:
import plotly.express as px

fig = px.line(
    rows,
    x="timestamp",
    y=[c for c in rows.columns if c != "timestamp"],
    title="Forecast + Quantile Bands"
)
fig


In [ ]:
import pandas as pd
import plotly.express as px

tmp = rows.copy()
tmp["hour"] = pd.to_datetime(tmp["timestamp"]).dt.hour
hourly = tmp.groupby("hour")["p50"].mean().reset_index()

fig = px.bar(hourly, x="hour", y="p50", title="Avg Forecast by Hour")
fig


In [ ]:
import plotly.express as px

fig = px.histogram(rows, x="p50", nbins=40, title="Forecast Distribution")
fig


In [ ]:
import pandas as pd
import plotly.express as px

tmp = rows.copy()
tmp["hour"] = pd.to_datetime(tmp["timestamp"]).dt.hour
tmp["bucket"] = pd.cut(tmp["hour"], bins=[0,6,12,18,24], labels=["Night","Morning","Afternoon","Evening"], right=False)

fig = px.box(tmp, x="bucket", y="p50", title="Forecast Spread by Time of Day")
fig


In [ ]:
import pandas as pd
import plotly.express as px

tmp = rows.copy()
tmp["hour"] = pd.to_datetime(tmp["timestamp"]).dt.hour

heat = tmp.groupby("hour")[["p50"]].mean().reset_index()
fig = px.imshow(
    heat.set_index("hour").T,
    aspect="auto",
    title="Heatmap: Avg Forecast by Hour"
)
fig


## Visual Sanity Checks

These plots provide a quick visual validation of recent behavior.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

repo_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
features_path = repo_root / 'data' / 'processed' / 'features.parquet'
if not features_path.exists():
    print('features.parquet not found. Run the feature pipeline first.')
else:
    df_viz = pd.read_parquet(features_path).sort_values('timestamp')
    if {'load_mw','wind_mw','solar_mw'}.issubset(df_viz.columns):
        recent = df_viz.tail(7 * 24)
        fig, ax = plt.subplots(3, 1, figsize=(12, 7), sharex=True)
        recent.plot(x='timestamp', y='load_mw', ax=ax[0], color='#1f77b4', title='Load (last 7 days)')
        recent.plot(x='timestamp', y='wind_mw', ax=ax[1], color='#2ca02c', title='Wind (last 7 days)')
        recent.plot(x='timestamp', y='solar_mw', ax=ax[2], color='#ff7f0e', title='Solar (last 7 days)')
        plt.tight_layout()
    else:
        print('Expected columns not found in features.parquet')
